In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np

import pandas as pd
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
dataframe_items=pd.read_csv("Dataset/data_ICM_metadata.csv",low_memory=True)
icm=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:
from src.Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    params={
    'n_components' : trial.suggest_int('no_components', 20, 200),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
    'item_alpha' : trial.suggest_float('item_alpha', 1e-6, 1e-1, log=True),
    'user_alpha' : trial.suggest_float('user_alpha', 1e-6, 1e-1, log=True),
    'epochs' : trial.suggest_int('epochs', 10, 50),
    'sgd_mode' : trial.suggest_categorical('sgd_mode', ['adagrad', 'adadelta']),
    'loss' : trial.suggest_categorical('loss', ['bpr', 'warp', 'warp-kos'])
    }



    model = LightFMCFRecommender(URM_train=URM_train)
    model.fit(**params, num_threads=4)


    result_df, _ = evaluator.evaluateRecommender(model)
    return result_df.loc[10, "MAP"]

In [ ]:
import optuna
from optuna.samplers import RandomSampler

database_path='hypertuning_results/tuning_lightFM.db'

study = optuna.load_study(study_name='hyperparameters_lightFM_cf',
                                storage=f'sqlite:///{database_path}')
study.sampler=RandomSampler(seed=SEED)


study.optimize(objective, n_trials=60, n_jobs=4)